In [1]:
import pandas as pd

import numpy as np
import re
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

In [2]:
full_df = pd.read_csv('../data/cleaned_reviews_data.csv', index_col=0)
full_df.shape

(8319, 26)

In [22]:
full_df.fillna('N/A', inplace=True)

In [23]:
with open('full_df.pkl', 'wb') as wf:
    pickle.dump(full_df, wf)

### By Description

In [ ]:
# categories_by_descriptions = {
#     'topic_0': 'cheap, affordable, and good value',
#     'topic_1': 'whiskey, bourbon, gin, rum, tequila',
#     'topic_2': 'red blend, merlot, cabernet sauvignon',
#     'topic_3': 'red pinot noir',
#     'topic_4': 'sweet cider',
#     'topic_5': 'white chardonnay',
#     'topic_6': 'beer',
#     'topic_7': 'white sauvignon blanc',
#     'topic_8': 'rose',
#     'topic_9': 'vodka and spirits'
# }

In [3]:
with open('topics_by_description.pkl','rb') as rf:
    topics_by_description_df = pickle.load(rf)


topics_by_description_df = full_df.merge(topics_by_description_df, on='Description')

topics_by_description_df.head()

,Name,Description,Category,Country,Alcohol_Vol,Review,Style,Aroma,Flavor,Sweetness,Bitterness,Enjoy,Pairing,Cocktails,Bottom_Line,Points,Price,Points_Ranking,Price_Range,Extend_Review,Photo_Link,Medal,Bands,Medal_Rank,Bands_Rank,Photo_Link_2,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,Mezcal de Leyendas Verde San Luis Potosi Mezca...,Mezcal de Leyendas Review Verde San Luis Potos...,Blanco Mezcal,Mexico,45%,"Clear color. Aromas and flavors of cucumber, m...","Fruity, Funky, Rich, Spicy, Smoky, Herbal & Co...","cucumber-melon, lime spritz, caramelized orang...","Same as aromas with notes of pickled ginger, s...",NaN,NaN,"in cocktails, on the rocks and neat",NaN,"Paloma, Sangrita, Tommy's Margarita","A sweet, savory, salty, juicy display of compl...",97.0,49.0,High,Mid Range,Blanco mezcal is a spirit from Mexico that can...,https://www.tastings.com/images-BTI/Glasses/Sp...,Platinum Medal,Superlative,5.0,5.0,https://www.tastings.com/Product-Images/Spirit...,0.0,0.008940,0.0,0.0,0.002073,0.0,0.000000,0.000000,0.000434,0.001905,0.005399,0.006336,0.011997,0.072079,0.005580,0.005652,0.008307,0.001893,0.0,0.016913
1,El Tesoro Paradiso Tequila Extra Anejo Mexico ...,El Tesoro Review Paradiso Tequila Extra Anejo ...,Extra Añejo Tequila,Mexico,40%,"Gold color. Inviting aromas of roasted nuts, c...","Fruity, Complex, Candied, Herbal, Rich & Spicy","roasted nuts, candied cherries, vanilla bean, ...","ripe melons, toffee, and white pepper",NaN,NaN,"in cocktails, neat, on the rocks and with cigars",NaN,"Paloma, Sangrita, Tommy's Margarita",A tantalizing melange of complex flavors that ...,97.0,180.0,High,Expensive,An extra añejo tequila must be aged for at lea...,https://www.tastings.com/images-BTI/Glasses/Sp...,Platinum Medal,Superlative,5.0,5.0,https://www.tastings.com/Product-Images/Spirit...,0.0,0.010294,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000411,0.000000,0.186663,0.019630,0.000000,0.007513,0.000000,0.0,0.000000
2,Peloton de la Muerte Vegan Pechuga Mezcal Mexi...,Peloton de la Muerte Review Vegan Pechuga Mezc...,Flavored Mezcal,Mexico,45.10%,Silvery straw color. Citrusy aromas and flavor...,"Complex, Fruity & Herbal","pickled melon and pineapple, candied jalapeño,...",Same as aromas with suggestions of curry,NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A joyous, extroverted Mezcal with juicy fruit ...",96.0,44.0,High,Mid Range,A Flavored Mezcal is an agave-based distillate...,https://www.tastings.com/images-BTI/Glasses/Sp...,Platinum Medal,Superlative,5.0,5.0,https://www.tastings.com/Product-Images/Spirit...,0.0,0.007432,0.0,0.0,0.000316,0.0,0.000752,0.000495,0.008162,0.009766,0.006304,0.004634,0.014264,0.055776,0.002775,0.008719,0.000000,0.001924,0.0,0.017815
3,El Tesoro Anejo Tequila Mexico Spirits,El Tesoro Review Anejo Tequila Mexico Mexican ...,Añejo Tequila,Mexico,40%,Straw color. Inviting aromas and flavors of ho...,"Fruity, Spicy & Herbal","honey-drizzled grapefruit, sweet apple compote...","Same as aromas with notes of pink peppercorns,...",NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A stylish, peppery tequila with an authentic v...",96.0,85.0,High,Expensive,"Añejo (""old"") Tequila is aged in wooden barrel...",https://www.tastings.com/images-BTI/Glasses/Sp...,Platinum Medal,Superlative,5.0,5.0,https://www.tastings.com/Product-Images/Spirit...,0.0,0.003658,0.0,0.0,0.002095,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002116,0.000000,0.222797,0.008191,0.000000,0.015395,0.000000,0.0,0.000000
4,1800 Milenio Extra Anejo Tequila Mexico Spirits,1800 Review Milenio Extra Anejo Tequila Mexico...,Extra Añejo Tequila,Mexico,40%,"Amber color. Aromas of toffee coated popcorn, ...","Fruity, Rich & Spicy","toffee coated popcorn, peanuts, warm madeira c...","allspice, mint tea, candied ginger,

In [4]:
with open('topics_by_description_matrix.pkl', 'rb') as rf:
    topics_by_description_matrix = pickle.load(rf)
with open('tfidf_description.pkl', 'rb') as rf:
    tfidf_des = pickle.load(rf)
with open('nmf_description.pkl', 'rb') as rf:
    nmf_des = pickle.load(rf)

In [24]:
test_item = ['cheap scotch']
topic_prob_dist = nmf_des.transform(tfidf_des.transform(test_item))
list_top_items_by_indices = list(cosine_similarity(topic_prob_dist, topics_by_description_matrix).argsort())[0][-1:-6:-1]
topics_by_description_df.iloc[list_top_items_by_indices]

,Name,Description,Category,Country,Alcohol_Vol,Review,Style,Aroma,Flavor,Sweetness,Bitterness,Enjoy,Pairing,Cocktails,Bottom_Line,Points,Price,Points_Ranking,Price_Range,Extend_Review,Photo_Link,Medal,Bands,Medal_Rank,Bands_Rank,Photo_Link_2,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
1563,Bird Dog Honey Jalapeno Flavored Whiskey USA S...,Bird Dog Review Honey Jalapeno Flavored Whiske...,Flavored Whiskey,USA,40%,Amber color. Aromas of beeswax and chamomile w...,Rich,beeswax and chamomile,"white flowers, honey on biscuits, caramel, and...",NaN,NaN,in cocktails,NaN,"Sazerac, Old Fashioned, Manhattan",A sweet and spicy whiskey that will be a fun c...,88.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.028308,0.105094,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.015191,0.0,0.013912,0.000000,0.000000,0.0,0.002380,0.018178,0.001911,0.0,0.002941
1564,Bird Dog Chocolate Flavored Whiskey USA Spirits,Bird Dog Review Chocolate Flavored Whiskey KY ...,Flavored Whiskey,USA,40%,Amber color. Confected aromas of caramel corn ...,Candied,caramel corn and chocolate frosting,chocolate sprinkles and tootsie rolls,NaN,NaN,in cocktails,NaN,"Irish Coffee, Old Fashioned, Manhattan","A sweet, chocolatey flavored whiskey that's ea...",88.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.026280,0.100866,0.0,0.0,0.000000,0.000000,0.005160,0.0,0.000000,0.015608,0.0,0.000718,0.000000,0.000000,0.0,0.002401,0.019983,0.000000,0.0,0.016433
5862,Bird Dog Apple Whiskey USA Spirits,Bird Dog Review Apple Whiskey KY Kentucky USA ...,Flavored Whiskey,USA,40%,Amber color. Aromas of green apple jolly ranch...,Candied & Fruity,green apple jolly rancher and caramel-apple pop,"caramel candies, kettle corn, honey roasted nu...",NaN,NaN,neat,NaN,NaN,A sweet and fruity flavored whiskey that will ...,85.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.001919,0.102182,0.0,0.0,0.007579,0.009654,0.003975,0.0,0.000000,0.012083,0.0,0.003112,0.000000,0.002315,0.0,0.005137,0.032653,0.000784,0.0,0.011999
3653,Bird Dog Black Cherry Flavored Whiskey USA Spi...,Bird Dog Review Black Cherry Flavored Whiskey ...,Flavored Whiskey,USA,40%,Golden amber color. Aromas and flavors of mara...,Candied & Fruity,"maraschino cherry, marzipan, and apricot stone",Same as aromas with accents of almond cookie,NaN,NaN,in cocktails,NaN,"Sazerac, Old Fashioned, Manhattan",The amaretto alternative for whiskey-lovers; d...,NaN,19.0,NaN,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Bronze Medal,Recommended,2.0,2.0,https://www.tastings.com/Product-Images/Spirit...,0.002258,0.098685,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.015281,0.0,0.005651,0.000000,0.000000,0.0,0.004259,0.030614,0.000000,0.0,0.017049
1582,Bird Dog Ruby Red Grapefruit Flavored Whiskey ...,Bird Dog Review Ruby Red Grapefruit Flavored W...,Flavored Whiskey,USA,40%,Golden amber color. Citrusy aromas of orange z...,Fruity & Candied,"orange zest, hints of caramel, and grapefruit ...",tangerine and orange hi-c,NaN,NaN,in cocktails and neat,NaN,"Sazerac, Old Fashioned, Manhattan",Bright citrus flavor pops on the palate of thi...,86.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.003968,0.099192,0.0,0.0,0.000000,0.000000,0.00

In [25]:
top_items = topics_by_description_df.iloc[list_top_items_by_indices].sample(3)
top_items

,Name,Description,Category,Country,Alcohol_Vol,Review,Style,Aroma,Flavor,Sweetness,Bitterness,Enjoy,Pairing,Cocktails,Bottom_Line,Points,Price,Points_Ranking,Price_Range,Extend_Review,Photo_Link,Medal,Bands,Medal_Rank,Bands_Rank,Photo_Link_2,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
5862,Bird Dog Apple Whiskey USA Spirits,Bird Dog Review Apple Whiskey KY Kentucky USA ...,Flavored Whiskey,USA,40%,Amber color. Aromas of green apple jolly ranch...,Candied & Fruity,green apple jolly rancher and caramel-apple pop,"caramel candies, kettle corn, honey roasted nu...",NaN,NaN,neat,NaN,NaN,A sweet and fruity flavored whiskey that will ...,85.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.001919,0.102182,0.0,0.0,0.007579,0.009654,0.003975,0.0,0.000000,0.012083,0.0,0.003112,0.000000,0.002315,0.0,0.005137,0.032653,0.000784,0.0,0.011999
1582,Bird Dog Ruby Red Grapefruit Flavored Whiskey ...,Bird Dog Review Ruby Red Grapefruit Flavored W...,Flavored Whiskey,USA,40%,Golden amber color. Citrusy aromas of orange z...,Fruity & Candied,"orange zest, hints of caramel, and grapefruit ...",tangerine and orange hi-c,NaN,NaN,in cocktails and neat,NaN,"Sazerac, Old Fashioned, Manhattan",Bright citrus flavor pops on the palate of thi...,86.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.003968,0.099192,0.0,0.0,0.000000,0.000000,0.009166,0.0,0.000966,0.023921,0.0,0.009107,0.000227,0.000000,0.0,0.001272,0.032577,0.000000,0.0,0.016505
1564,Bird Dog Chocolate Flavored Whiskey USA Spirits,Bird Dog Review Chocolate Flavored Whiskey KY ...,Flavored Whiskey,USA,40%,Amber color. Confected aromas of caramel corn ...,Candied,caramel corn and chocolate frosting,chocolate sprinkles and tootsie rolls,NaN,NaN,in cocktails,NaN,"Irish Coffee, Old Fashioned, Manhattan","A sweet, chocolatey flavored whiskey that's ea...",88.0,19.0,Low,Cheap,Flavored whiskeys are distilled alcoholic beve...,https://www.tastings.com/images-BTI/Glasses/Sp...,Silver Medal,Highly Recommended,3.0,3.0,https://www.tastings.com/Product-Images/Spirit...,0.026280,0.100866,0.0,0.0,0.000000,0.000000,0.005160,0.0,0.000000,0.015608,0.0,0.000718,0.000000,0.000000,0.0,0.002401,0.019983,0.000000,0.0,0.016433


In [27]:

top_items.iloc[0].Bottom_Line

'A sweet and fruity flavored whiskey that will be great for folks seeking a caramel-apple shooter.'

In [34]:
full_df.Bottom_Line

0       A sweet, savory, salty, juicy display of compl...
1       A tantalizing melange of complex flavors that ...
2       A joyous, extroverted Mezcal with juicy fruit ...
3       A stylish, peppery tequila with an authentic v...
4       A richly flavored Extra Añejo Tequila with an ...
                              ...                        
8314    A bone dry, adventurous cider for kombucha lov...
8315                   A sweet, subtle apple-y refresher.
8316    Pear notes shine in this cider that is sure to...
8317    Attractive tannins make this a solid choice fo...
8318                          A rustic cider with issues.
Name: Bottom_Line, Length: 8319, dtype: object